#I'll walk you through this code step by step, explaining what each part does and why it's needed. Think of this as building a system to collect, store, and retrieve stock market data.Line-by-Line Code ExplanationDocument # Stock Data Management System - Line by Line Explanation




**What's haThe code essentially creates a personal stock data warehouse. Here's the simple workflow:

Input: List of 50 company names from NIFTY 50
Process: Download their stock prices, clean the data, store in database
Output: Fast access to historical stock data whenever you need it

Think of it like having your own personal Bloomberg terminal, but built with Python! You collect the data once, then you can analyze it as many times as you want without waiting for internet downloads.
The beauty of this system is that once you run fetch_and_store_all() once, you have all the data locally. Then you can run analysis, create charts, or build trading strategies without depending on internet connectivity or API limits.
Would you like me to explain any specific part in more detail, or do you have questions about how any particular section works?

In [1]:
# -----------------------------------------
# Data Storage - SQLite
import pandas as pd
import os
from pathlib import Path
from quantmod.markets import getData
from sqlalchemy import create_engine, text


def get_stock_info():
    stock_table = pd.read_csv("ind_nifty50list.csv")
    return stock_table


symbols = get_stock_info().Symbol.to_list()
nifty50 = [symbol + ".NS" for symbol in symbols]

# Create database directory if it doesn't exist
db_dir = Path("database")
db_dir.mkdir(parents=True, exist_ok=True)

# Connect to SQLite database (using local path)
engine = create_engine("sqlite:///database/equities2.db", echo=False)


# SQL query to create table if not exists
def create_table():
    create_table_query = """
    CREATE TABLE IF NOT EXISTS stock_data (
        ticker TEXT NOT NULL,
        date TEXT NOT NULL,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER,
        PRIMARY KEY (ticker, date)
    );
    """

    # Execute the table creation
    with engine.connect() as conn:
        conn.execute(text(create_table_query))
        conn.commit()  # Add explicit commit
        print("Table created successfully.")


# Run once and create the table
create_table()

# -----------------------------------------
# Fetch and Store Function


# Fetch data and store in SQLite
def fetch_and_store_all():
    for symbol in nifty50:
        try:
            (
                getData(symbol, period="250d")
                .reset_index()
                .assign(ticker=symbol.replace(".NS", ""))
                .loc[:, ["ticker", "Date", "Open", "High", "Low", "Close", "Volume"]]
                .rename(columns=str.lower)
                .to_sql("stock_data", engine, if_exists="append", index=False)
            )
            print(f"Saved: {symbol}")
        except Exception as e:
            print(f"Error with {symbol}: {e}")


# Run once and populate the database
fetch_and_store_all()


# -----------------------------------------
# Query: All Stocks
def query_all_stocks(engine=engine):
    query = text("""
        SELECT ticker, date, close
        FROM stock_data
        ORDER BY ticker, date
    """)

    with engine.connect() as conn:
        data = pd.read_sql(query, conn)

    # Preprocess
    data["date"] = pd.to_datetime(data["date"])
    data.set_index("date", inplace=True)

    # Pivot: rows = date, columns = ticker, values = close
    data = data.pivot(columns="ticker", values="close")

    # Fill missing values for JIOFIN
    if "JIOFIN" in data.columns:
        data["JIOFIN"] = data["JIOFIN"].ffill().bfill()

    # Remove 'ETERNAL' from the DataFrame
    data = data.drop(columns="ETERNAL", errors="ignore")

    return data


# -----------------------------------------
# Query from Database
def query_stock(ticker, engine=engine):
    query = text("SELECT * FROM stock_data WHERE ticker = :ticker ORDER BY date")

    with engine.connect() as conn:
        data = pd.read_sql(query, conn, params={"ticker": ticker})

    # Convert the date column to datetime format
    data["date"] = pd.to_datetime(data["date"])

    # Set the date column as the DataFrame index
    data.set_index("date", inplace=True)

    return data



Table created successfully.
Saved: ADANIENT.NS
Saved: ADANIPORTS.NS
Saved: APOLLOHOSP.NS
Saved: ASIANPAINT.NS
Saved: AXISBANK.NS
Saved: BAJAJ-AUTO.NS
Saved: BAJFINANCE.NS
Saved: BAJAJFINSV.NS
Saved: BEL.NS
Saved: BHARTIARTL.NS
Saved: CIPLA.NS
Saved: COALINDIA.NS
Saved: DRREDDY.NS
Saved: EICHERMOT.NS
Saved: ETERNAL.NS
Saved: GRASIM.NS
Saved: HCLTECH.NS
Saved: HDFCBANK.NS
Saved: HDFCLIFE.NS
Saved: HEROMOTOCO.NS
Saved: HINDALCO.NS
Saved: HINDUNILVR.NS
Saved: ICICIBANK.NS
Saved: ITC.NS
Saved: INDUSINDBK.NS
Saved: INFY.NS
Saved: JSWSTEEL.NS
Saved: JIOFIN.NS
Saved: KOTAKBANK.NS
Saved: LT.NS
Saved: M&M.NS
Saved: MARUTI.NS
Saved: NTPC.NS
Saved: NESTLEIND.NS
Saved: ONGC.NS
Saved: POWERGRID.NS
Saved: RELIANCE.NS
Saved: SBILIFE.NS
Saved: SHRIRAMFIN.NS
Saved: SBIN.NS
Saved: SUNPHARMA.NS
Saved: TCS.NS
Saved: TATACONSUM.NS
Saved: TATAMOTORS.NS
Saved: TATASTEEL.NS
Saved: TECHM.NS
Saved: TITAN.NS
Saved: TRENT.NS
Saved: ULTRACEMCO.NS
Saved: WIPRO.NS


In [ ]:
import sqlite3

conn = sqlite3.connect("stock_data.db")
cursor = conn.cursor()